In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification, BertTokenizerFast
import numpy as np
import torch
from sklearn import preprocessing
import torch.nn as nn
from sklearn.metrics import classification_report
import tensorflow as tf
tf.config.experimental.set_memory_growth
import os
from transformers import get_linear_schedule_with_warmup

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:2048"
device = torch.device("cpu")


/home/user3/miniconda3/envs/venv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-01 10:45:22.269989: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_set = pd.read_csv("train.csv", encoding= 'unicode_escape', sep = ',')
test_set = pd.read_csv("test.csv", encoding= 'unicode_escape', sep = ',')
train_set = train_set.fillna('')
test_set = test_set[:3534]  #trailing empty rows
print(len(train_set), len(test_set))

27481 3534


In [3]:
X_train = train_set['text']
y_train = train_set['sentiment']
y_train = y_train.replace(['negative','neutral','positive'],[0,1,2]) 
X_test = test_set['text']
y_test = test_set['sentiment']
y_test = y_test.replace(['negative','neutral','positive'],[0,1,2]) 


In [4]:
bert = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 3, output_attentions = False, output_hidden_states = False)
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', do_lower_case = True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [5]:
train_tokens = tokenizer.batch_encode_plus(
    X_train.tolist(),
    add_special_tokens = True,
    max_length = 50,
    padding='max_length',
    truncation=True,
    return_attention_mask = True
)
test_tokens = tokenizer.batch_encode_plus(
    X_test.tolist(),
    add_special_tokens = True,
    max_length = 50,
    padding='max_length',
    truncation=True,
    return_attention_mask = True,
)

In [6]:
train_seq = torch.tensor(train_tokens['input_ids'])
train_mask = torch.tensor(train_tokens['attention_mask'])
train_y = torch.tensor(y_train.tolist())

test_seq = torch.tensor(test_tokens['input_ids'])
test_mask = torch.tensor(test_tokens['attention_mask'])
test_y = torch.tensor(y_test.tolist())


In [7]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 32

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
test_data = TensorDataset(test_seq, test_mask, test_y)

# sampler for sampling the data during training
test_sampler = SequentialSampler(test_data)

# dataLoader for validation set
test_dataloader = DataLoader(test_data, sampler = test_sampler, batch_size=batch_size)


In [8]:
bert.to(device)
print(bert)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [9]:

# define the optimizer
optimizer = torch.optim.AdamW(bert.parameters(), lr = 5e-5, eps = 1e-8)


In [10]:
epochs = 4
total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

In [25]:
def accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [12]:
training_stats = []
best = float('-inf')
# Measure the total training time for the whole run.

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. 
    bert.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 100 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

        # Unpack this training batch from our dataloader. 
        # 
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Clear Gradients
        bert.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch)
        output = bert(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        loss, logits = output.loss, output.logits

        # Accumulate the training loss over all of the batches
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(bert.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")


    # Put the model in evaluation mode
    bert.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in test_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            output = bert(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
            loss, logits = output.loss, output.logits
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_test_accuracy = total_eval_accuracy / len(test_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_test_accuracy))
    if avg_test_accuracy > best:
        torch.save(bert.state_dict(), 'weights.pt')
    # Calculate the average loss over all of the batches.
    avg_test_loss = total_eval_loss / len(test_dataloader)
    
    # Measure how long the validation run took.
    
    print("  Validation Loss: {0:.2f}".format(avg_test_loss))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_test_loss,
            'Valid. Accur.': avg_test_accuracy,
        }
    )

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch   100  of    859.
  Batch   200  of    859.
  Batch   300  of    859.
  Batch   400  of    859.
  Batch   500  of    859.
  Batch   600  of    859.
  Batch   700  of    859.
  Batch   800  of    859.

  Average training loss: 0.57

Running Validation...
  Accuracy: 0.79
  Validation Loss: 0.51

======== Epoch 2 / 4 ========
Training...
  Batch   100  of    859.
  Batch   200  of    859.
  Batch   300  of    859.
  Batch   400  of    859.
  Batch   500  of    859.
  Batch   600  of    859.
  Batch   700  of    859.
  Batch   800  of    859.

  Average training loss: 0.38

Running Validation...
  Accuracy: 0.80
  Validation Loss: 0.52

======== Epoch 3 / 4 ========
Training...
  Batch   100  of    859.
  Batch   200  of    859.
  Batch   300  of    859.
  Batch   400  of    859.
  Batch   500  of    859.
  Batch   600  of    859.
  Batch   700  of    859.
  Batch   800  of    859.

  Average training loss: 0.22

Running Validation...
  A

In [12]:

path = 'weights.pt'
bert.load_state_dict(torch.load(path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [27]:
bert.eval()

total_eval_accuracy = 0

for batch in test_dataloader:

    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    
    with torch.no_grad():        

        output = bert(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask,
                                labels=b_labels)
        
        logits = output.logits
        

    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    total_eval_accuracy += accuracy(logits, label_ids)
    

avg_test_accuracy = total_eval_accuracy / len(test_dataloader)
print("  Accuracy: {0:.2f}".format(avg_test_accuracy))




  Accuracy: 0.79
